In [12]:
import pandas as pd
import os
import re
import numpy as np
import logging
logger = logging.getLogger(__name__)

In [2]:
current_directory = os.getcwd()
data_directory = os.path.join(current_directory, r'data')
data_directory

'c:\\Users\\petrovdu\\OneDrive - Organon\\documents\\diabetes-batut\\data'

File routines

In [22]:
logger.setLevel(logging.INFO)
logger.info('Started')


regex = re.compile(r'\d+')

def extract_okrug(text):
    pattern = r'\b(\w+\s+okrug|grad\s+Beograd)\b'
    if pd.notna(text):
        match = re.search(pattern, text)
        if match:
            return match.group()
    return None    

atc_prop_df = pd.DataFrame()
atc_real_df = pd.DataFrame()
mtp_real_df = pd.DataFrame()
mtp_prop_df = pd.DataFrame()

for (dirpath, dirnames, filenames) in os.walk(data_directory):
    for file in filenames:
        file_path = os.path.join(dirpath, file)

        if file.endswith('.csv'):
            logger.info(f"Reading File {file}")
            df = pd.read_csv(data_directory+'\\' + file ,sep = ";", encoding='windows-1250',on_bad_lines='skip')
            # add year column
            year = regex.findall(file)[0] 
            df['Godina'] = year  
            # extract okrug from filijala   
            df['Okrug'] = df['Filijala'].apply(lambda x: extract_okrug(x))    
            # drop Filijala
            df.drop(['Filijala'], axis=1,inplace=True)
            # fill empty okrug
            df['Okrug'] = df.Okrug.fillna('Nepoznat')
            # group files by filename
            base_filename = re.sub('\d+','', file)
            
            if base_filename == 'atc_propisani.csv':
                atc_prop_df = pd.concat([atc_prop_df, df], ignore_index=True)

            elif base_filename == 'atc_realizovani.csv':
                atc_real_df = pd.concat([atc_real_df, df], ignore_index=True)

            elif base_filename == 'mtp_realizovani.csv':
                mtp_real_df = pd.concat([mtp_real_df, df], ignore_index=True)


            elif base_filename == 'mtp_propisani.csv':
                mtp_prop_df = pd.concat([mtp_prop_df, df], ignore_index=True)   

            else:
                logger.info (f"Skipping unsupported file: {file}")

                     
                
        elif file.endswith('.xlsx'):
            logger.info(f"Reading File {file}")
            if file == 'filijale.xlsx':
                filijale = pd.read_excel(data_directory+'\\' + file)
            if file == 'sifremtp.xlsx':
                sifremtp = pd.read_excel(data_directory+'\\' + file)
                
        else:
            logger.info(f"Skipping unsupported file: {file}")

mtp_real_df['SifraPomagala'] =  np.nanmax(mtp_real_df[['SifraPomagala','SifraLeka']].values , axis=1)
mtp_real_df.drop(['SifraLeka'], axis=1,inplace=True)

# Save concatenated DataFrames

atc_prop_df.to_csv(os.path.join(current_directory, 'atc_prop.csv'),encoding='windows-1250', index=False)
atc_real_df.to_csv(os.path.join(current_directory, 'atc_real.csv'),encoding='windows-1250', index=False)
mtp_real_df.to_csv(os.path.join(current_directory, 'mtp_real.csv'),encoding='windows-1250', index=False)
mtp_prop_df.to_csv(os.path.join(current_directory, 'mtp_prop.csv'),encoding='windows-1250', index=False)

logger.info(f"Saved files in: {current_directory}")


logger.info('Finished')

In [14]:
mtp_real_df.dtypes

IdHash                    int64
Filijala                 object
Pol                      object
Uzrast                    int64
BrojIzdatihPakovanja      int64
Godina                   object
Okrug                    object
SifraPomagala           float64
dtype: object

In [16]:
mtp_real_df.tail()

,IdHash,Filijala,Pol,Uzrast,BrojIzdatihPakovanja,Godina,Okrug,SifraPomagala
1758368,8407479,Filijala za Južnobacki okrug sa sedištem u Nov...,F,70,90,2021,Južnobacki okrug,14710.0
1758369,4713803,Filijala za grad Beograd,M,68,300,2021,grad Beograd,15020.0
1758370,8047680,Filijala za grad Beograd,M,84,50,2021,grad Beograd,15040.0
1758371,8047680,Filijala za grad Beograd,M,84,90,2021,grad Beograd,14710.0
1758372,4164866,Filijala za Južnobanatski okrug sa sedištem u ...,M,59,50,2021,Južnobanatski okrug,15040.0


In [24]:
! jupyter nbconvert --to script etl.ipynb

[NbConvertApp] Converting notebook etl.ipynb to script
[NbConvertApp] Writing 2818 bytes to etl.py
